## InstructLab Walkthrough

The purpose of this repository is to demonstrate how to perform RAFT (Retrieval Augmented Fine Tuning) by using InstructLab & Milvus on RHEL.

Fine Tuning: Train Llama-3.2-1B to know about LLaMA 4 by using InstructLab's Synthetic Data Generation.

Serving: The fine tuned model is served via vLLM on top of InstructLab.

Retrieval Augmented Generation: Integrate organizational context into the fine-tuned model by embedding the document located at rag/DOG.md into Milvus. This document defines appropriate and inappropriate use cases for LLaMA 4 within a dog adoption organization. The Chat bot will answer according to the organizational context.

**This has been tested on a Notebook with a single Nvidia L4 GPU**

Install InstructLab -

In [ ]:
pip install instructlab==0.26.1

Install llama-cpp-python in order to utilze the GPU in the system.

**This might fail, but this is OK**

In [ ]:
!CMAKE_ARGS="-DGGML_CUDA=on" \
  FORCE_CMAKE=1 \
  pip install --no-cache-dir --force-reinstall llama-cpp-python==0.3.6

In [ ]:
!pip install numpy==1.26.4 instructlab-training[cuda]

## Verify the installation of InstructLab

In [ ]:
!ilab --version

In [ ]:
!ilab system info

In [ ]:
!nvidia-smi

## Configuring Parameters

In [ ]:
HF_TOKEN="<Insert Token>"

In [ ]:
!ilab config init --non-interactive

In [ ]:
!ilab model download

In [ ]:
!mkdir -p ~/.local/share/instructlab/taxonomy/knowledge/llms/llama/

!cp ../sdg/qna.yaml ~/.local/share/instructlab/taxonomy/knowledge/llms/llama/

In [ ]:
!ilab taxonomy diff

## Generate Data

In [ ]:
!ilab data generate --num-instructions 500 --enable-serving-output --gpus 1

## Train Model

In [ ]:
!ilab model download --repository=meta-llama/Llama-3.2-1B-Instruct --hf-token $HF_TOKEN

In [ ]:
!ilab model train --model-path ~/.cache/instructlab/models/meta-llama/Llama-3.2-1B-Instruct --data-path ~/.local/share/instructlab/datasets/2025-06-16_121930/knowledge_train_msgs_2025-06-16T12_19_38.jsonl --device cuda --pipeline accelerated --gpus 1 --num-epochs 15

## Chat with Model

In [ ]:
!ilab model serve --model-path /home/instruct/.cache/instructlab/models/meta-llama/Llama-3.2-1B-Instruct/ --gpus=1

## Upload Model

In [ ]:
pip install boto3

In [ ]:
import boto3, os, pathlib

model_dir = pathlib.Path("/opt/app-root/src/.cache/instructlab/models/instructlab/granite-7b-lab")   # where your .bin / safetensors live
bucket     = "models"
prefix     = "granite-7b/"

s3 = boto3.client(
        "s3",
        endpoint_url="url",
        aws_access_key_id='minioadmin',
        aws_secret_access_key='minioadmin')

for f in model_dir.iterdir():
    if f.is_file():
        s3.upload_file(str(f), bucket, prefix + f.name)